In [1]:
import numpy as np

from tabulate import tabulate
from IPython.display import clear_output
from experiment_CM3 import *
from newinstruments.nwa2 import *
from newinstruments.instrumenttypes import *

import time
import numpy as np
import matplotlib.pyplot as plt

from scripts.openfiles import *
from tqdm import tqdm

#from IPython.display import display, HTML
#%config InlineBackend.figure_format='retina'

In [ ]:
from pymeasure.instruments.srs import sr830, sr844
from pymeasure.instruments.yokogawa import yokogawa7651, yokogawaGS200
from pymeasure.instruments.agilent import Agilent33500
from newinstruments.SignalHound import SignalHoundSA124B
from newinstruments.SignalCore import SignalCore
from newinstruments.BlueFors import BlueFors
from newinstruments.nwa2 import *
#from newinstruments.mcc_daq import *

In [4]:
vna = E5071_2('GPIB0::2::INSTR')

yoko_Lch     = yokogawaGS200.YokogawaGS200('GPIB1::7::INSTR')
yoko_pinch     = yokogawaGS200.YokogawaGS200('GPIB1::1::INSTR')
yoko_Lchgt    = yokogawaGS200.YokogawaGS200("GPIB1::21::INSTR")
yoko_gtAll    = yokogawaGS200.YokogawaGS200("GPIB1::6::INSTR")
yoko_res    = yokogawa7651.Yokogawa7651("GPIB1::3::INSTR")

filament    = Agilent33500("GPIB1::17::INSTR")
lockin_HF   = sr844.SR844('GPIB1::11::INSTR')
daq         = mcc_daq()

generator   = Agilent33500("GPIB1::19::INSTR")

bluefors    = BlueFors()

In [5]:
gain = 1e-6*1e9     # 1e-6 is 1uA/V, 1e9 is nA
GHz = 1e9
MHz = 1e6
kHz = 1e3

tau = 1.0
o14 = 0.0 
o23 = 0.0 

T6 = bluefors.get_temperature(6)
print(T6)

In [7]:
#generator.shape     = 'SIN'   
generator.frequency = 3*MHz              
generator.amplitude = 0.2
#generator.output    = 'on'               

In [8]:
lockin_HF.sensitivity   = 0.001  # 3e-4
lockin_HF.time_constant =  tau

print(lockin_HF.sensitivity) # in Volts

0.001


In [9]:
control_instr_dict = {
                    'Vch': [0, yoko_Lch, 'source_voltage', 'ramp_to_voltage', 'V'],
                    'Vpinch': [0, yoko_pinch, 'source_voltage', 'ramp_to_voltage', 'V'],
                    'VLchgt': [0, yoko_Lchgt, 'source_voltage', 'ramp_to_voltage', 'V'],
                    'VgtAll': [0, yoko_gtAll, 'source_voltage', 'ramp_to_voltage', 'V'],
                    'Vrs': [0.7, yoko_res, 'source_voltage', 'ramp_to_voltage', 'V'],
                    'Vac': [0.05, generator, 'amplitude', 'ramp_to_amplitude', 'Vpp'],
                    'fac': [3*MHz, generator, 'frequency', 'ramp_to_frequency', 'Hz']
}

vna_control_dict = {
                    'set_full' : [("cmTest", 'S21', 1), vna, 'set_full_measure'],
                    'format': [('MLOG',), vna, 'set_format'],  # Note the comma after 'MLOG'
                    'power' : [(-30,), vna, 'set_power'], 
                    'avg' : [("OFF",), vna, 'set_average_state'],  
                    'num_avg': [(5,), vna, 'set_averages'], 
                    'output': [(False,), vna, 'set_output'],  
                    'sweep_pts': [(5,), vna, 'set_sweep_points'],  
                    'electrical_delay': [(49.11e-9,), vna, 'set_electrical_delay'],  
                    'start_freq': [(3e9,), vna, 'set_start_frequency'],
                    'stop_freq': [(6e9,), vna, 'set_stop_frequency']


}

readout_instr_dict = {
                    'Vx': [lockin_HF, 'x', 'V'],
                    'Vy': [lockin_HF, 'y', 'V'],
                    'vna_freq' : [vna, 'get_fpoints', 'Hz'],
                    'vna_y1' : [vna, 'read_data_y1', 'U'],
                    'vna_y2' : [vna, 'read_data_y2', 'U']
}

In [10]:
exp = experiment_CM3(control_instr_dict, vna_control_dict, readout_instr_dict)

--------  -----------
V14       0
V23       0
Vac       0.05
Vch       0
Vgt       0
Vrs       0.7
comment   None
comment2  None
fac       3,000,000.0
tconst    0.1
--------  -----------


## 🏆 **1D scan / sweep** 

In [11]:
integration_time = 0.1

T6 = round(bluefors.get_temperature(6)*1e3)

vna_onoff = True
lockin_HF.time_constant =  integration_time

exp.Vac = 0.04
exp.fac = 3.0*MHz 
exp.Vch = 0.0 
exp.VLchgt = 0.0
exp.VgtAll = 0.0
exp.Vpinch = 0.0
exp.Vrs = 0.4
exp.tconst = integration_time
exp.comment = f'Tmxc={T6}mK / gain=+40dB '
exp.comment2 = f'vna output {rf_onoff} / vna added atten -20dB'

print(tabulate(exp.gettable_ClassAttributes()))

--------  ----------------------------------------
V14       0.0 / off=0.0
V23       0.0 / off=0.0
Vac       0.04
Vch       0.0
Vgt       0
Vrs       0.4
comment   Tmxc=550mK / gain=+40dB / sourceatt-20dB
comment2  None
fac       3,000,000.0
tconst    0.1
--------  ----------------------------------------


In [12]:
exp.instr_init()

instruments are initialized


In [13]:
#s1 & s2 are sweep start and stop, for HF measurements can go to log scale

exp.sweep_params(
    var     =   ['Vch'],
    s1      =   0.0, 
    s2      =   1.5,  
    num     =   71,
    scale   =   'linear',
    offset  =   [0]
)

print(tabulate(exp.gettable_ClassAttributes()))
print(exp.estimate_run_time(exp_type='1D', vna_type=True))

--------  ---------------------------------------------
V14       sweep / 0.0 : 1.5 / num=71 / linear / off=0.0
V23       sweep / 0.0 : 1.5 / num=71 / linear / off=0.0
Vac       0.04
Vch       sweep / 0.0 : 1.5 / num=71 / linear / off=0
Vgt       0
Vrs       0.4
comment   Tmxc=550mK / gain=+40dB / sourceatt-20dB
comment2  None
fac       3,000,000.0
tconst    0.1
--------  ---------------------------------------------
experiment time is approx 26 sec


#### 💬 **DONT FORGET TO CHANGE Experiment name**

In [14]:
exp_name = 'sweep_A0'

exp.run(exp_name, exp_type = '1D', vna_type = True, lockin_type = False, savedata=True)

100%|███████████████████████████████████████████████████████████████| 71/71 [00:25<00:00,  2.82it/s]

experiment is successfully finished
closed db


## 🎯 **2D scan / sweep / step**

In [15]:
o23 = 0.0
o14 = 0.0
integration_time = 0.1

T6 = round(bluefors.get_temperature(6)*1e3)

# rf_onoff = True
# print(f'SignalCore output will be {rf_onoff}')

lockin_HF.time_constant =  integration_time

# exp.hf_freq = round(5.5*GHz)
# exp.hf_pow = -25
# exp.sa_freq = round(5.5*GHz)
exp.Vac = 0.04
exp.fac = 3.0*MHz
exp.Vch = 0.0
exp.V14 = {'val': 0.0, 'off': o14}
exp.V23 = {'val': 0.0, 'off': o23}
exp.Vgt = 0
exp.Vrs = 0.4
exp.tconst = integration_time
exp.comment = f'Tmxc={T6}mK / gain=+40dB / sourceatt-20dB'
# exp.comment2 = f'RF output {rf_onoff} / RF-10dB / adding constriction'

print(tabulate(exp.gettable_ClassAttributes()))

--------  ----------------------------------------
V14       0.0 / off=0.0
V23       0.0 / off=0.0
Vac       0.04
Vch       0.0
Vgt       0
Vrs       0.4
comment   Tmxc=550mK / gain=+40dB / sourceatt-20dB
comment2  None
fac       3,000,000.0
tconst    0.1
--------  ----------------------------------------


In [16]:
exp.instr_init()

instruments are initialized


In [114]:
exp.sweep_params(
    var     =   ['Vch', 'V14', 'V23'],
    s1      =   0.0,
    s2      =   1.5,
    num     =   61,
    scale   =   'linear',
    offset  =   [0, o14, o23]
)

exp.step_params(
    var     =   'V23', #['hf_freq','sa_freq'],
    s1      =   0.2,
    s2      =   0.7,
    num     =   51,
    scale   =   'linear',
    offset  =   0
)
print(tabulate(exp.gettable_ClassAttributes()))
print(exp.estimate_run_time(exp_type='2D'))

--------  ---------------------------------------------
V14       sweep / 0.4 : 2.0 / num=71 / linear / off=0.0
V23       sweep / 0.4 : 2.0 / num=71 / linear / off=0.0
Vac       step / 0.2 : 0.7 / num=51 / linear / off=0
Vch       sweep / 0.4 : 2.0 / num=71 / linear / off=0
Vgt       0
Vrs       0.7
comment   Tmxc=551mK / gain=+40dB / sourceatt-20dB
comment2  RF-20dB / V23=Vch+step
fac       3,000,000.0
o23       step / -1.0 : 1.0 / num=41 / linear / off=0
tconst    0.1
--------  ---------------------------------------------
experiment time is approx 32.3 mins


In [115]:
exp_name = 'sweep_A1'
# sc.set_output_state(rf_onoff)
exp.run(exp_name, exp_type='2D')
# sc.set_output_state(False)

loop 51/51: 100%|███████████████████████████████████████████████████| 71/71 [00:25<00:00,  2.81it/s]

experiment is successfully finished
closed db


## 🏀 **set of 1D-scans with outer loop**

In [48]:
o23 = 0.0
o14 = 0.0
integration_time = 0.3

T6 = round(bluefors.get_temperature(6)*1e3)

# rf_onoff = True
# print(f'SignalCore output will be {rf_onoff}')

lockin_HF.time_constant =  integration_time

# exp.hf_freq = round(5.5*GHz)
# exp.hf_pow = -10
# exp.sa_freq = round(5.5*GHz)
exp.Vac = 0.04
exp.fac = 3.0*MHz
exp.Vch = 0.0
exp.V14 = {'val': 0.0, 'off': o14}
exp.V23 = {'val': 0.0, 'off': o23}
exp.Vgt = 0
exp.Vrs = 0.4
exp.tconst = integration_time
exp.comment = f'Tmxc={T6}mK / gain=+40dB / sourceatt-20dB'
# exp.comment2 = f'RF {rf_onoff} / RF-10dB'

print(tabulate(exp.gettable_ClassAttributes()))

--------  ----------------------------------------
V14       0.0 / off=0.0
V23       0.0 / off=0.0
Vac       0.04
Vch       0.0
Vgt       0
Vrs       0.4
comment   Tmxc=550mK / gain=+40dB / sourceatt-20dB
comment2  RF-20dB / V14=Vch+step-o14
fac       3,000,000.0
tconst    0.3
--------  ----------------------------------------


In [49]:
exp.instr_init()

instruments are initialized


In [50]:
step_control_parameter = 'd14'
step_list = np.linspace(-0.4, -0.1, num=6)
sweep_list = np.linspace(0.0, 1.0, num=121)

setattr(exp, step_control_parameter, 0)

exp.step_params(
    var     =   step_control_parameter,
    s1      =   step_list[0],
    s2      =   step_list[-1],
    num     =   len(step_list),
    scale   =   'linear',
    offset  =   0
)

exp.sweep_params(
    var     =   ['Vch', 'V14', 'V23'],
    s1      =   sweep_list[0],
    s2      =   sweep_list[-1],
    num     =   len(sweep_list),
    scale   =   'linear',
    offset  =   [0, o14, o23]
)

exp.comment2 = f'RF-20dB / V14=Vch+step-o14'

print(tabulate(exp.gettable_ClassAttributes()))
print(exp.estimate_run_time(exp_type='2D'))

--------  ----------------------------------------------
V14       sweep / 0.0 : 1.0 / num=121 / linear / off=0.0
V23       sweep / 0.0 : 1.0 / num=121 / linear / off=0.0
Vac       0.04
Vch       sweep / 0.0 : 1.0 / num=121 / linear / off=0
Vgt       0
Vrs       0.4
comment   Tmxc=550mK / gain=+40dB / sourceatt-20dB
comment2  RF-20dB / V14=Vch+step-o14
d14       step / -0.4 : -0.1 / num=6 / linear / off=0
fac       3,000,000.0
tconst    0.3
--------  ----------------------------------------------
experiment time is approx 13.7 mins


In [51]:
exp_name = 'sweep_A6c'
# sc.set_output_state(rf_onoff)

db = exp.create_sqldb(exp_name)

# exp.run(exp_name, exp_type='2D')
# sc.set_output_state(False)

for d14 in step_list:
    exp.sweep_params(
        var     =   ['Vch', 'V14', 'V23'],
    s1      =   sweep_list[0],
    s2      =   sweep_list[-1],
    num     =   len(sweep_list),
        scale   =   'linear',
        offset  =   [0, -1*d14-o14, o23]
    )
    exp.run_outerloop(sqldb=db, exp_name=exp_name)

exp.close_sqldb(db)
print('experiment is successfully finished')
delattr(exp, step_control_parameter)

# sc.set_output_state(False)

100%|█████████████████████████████████████████████████████████████| 121/121 [01:54<00:00,  1.06it/s]


experiment is successfully finished


## **Set of 1D scans**

## 🎻 **set of 2D scans**

In [29]:
hffreq_list = [4.22*GHz, 4.72*GHz, 5.22*GHz, 5.72*GHz, 6.47*GHz, 7.51*GHz, 7.65*GHz]

In [30]:
rf_onoff = True
sc.set_output_state(rf_onoff)

sweep_num_today = 

for i in range(len(hffreq_list)):

    T6 = round(bluefors.get_temperature(6)*1e3)
    
    exp.hf_freq = round(hffreq_list[i])
    exp.sa_freq = round(hffreq_list[i])
    exp.hf_pow = -25
    exp.Vac = 0.03
    exp.Vch = 0.25
    exp.V14 = {'val': 0.25, 'off': o14}
    exp.V23 = {'val': 0.25, 'off': o23}
    exp.Vgt = 0
    exp.Vrs = 0.7
    exp.tconst = 0.1
    exp.comment = f'Tmxc={T6}mK / gain=+40dB / sourceatt-20dB / HF_biastees / HF circuit setup / RF-10dB '
    exp.extra_comment = f'RF {rf_onoff}'

    exp.instr_init()

    exp.sweep_params(
        var     =   ['Vch', 'V23', 'V14'],
        s1      =   0.4,
        s2      =   1.7,
        num     =   131,
        scale   =   'linear',
        offset  =   [0, o23, o14]
    )

    exp.step_params(
        var     =   'hf_pow',
        s1      =   -25,
        s2      =   -5,
        num     =   20,
        scale   =   'linear',
        offset  =   0
    )
    exp_name = 'sweep_' + str(i+sweep_num_today)
    exp.run_2D(exp_name)

sleep(1)
sc.set_output_state(False)

loop 20/20: 100%|█████████████████████████████████████████████████| 131/131 [00:54<00:00,  2.39it/s]


experiment is successfully finished
closed db
SignalCore10003410 : Set output state to False


0

In [12]:
o14 = -0.3
o23 = -0.3

In [118]:
rf_onoff = True
sc.set_output_state(rf_onoff)
T6 = round(bluefors.get_temperature(6)*1e3)

def set_init(value):
    exp.Vac = 0.03
    exp.fac = 3.0*MHz
    exp.Vch = 0.15
    exp.V14 = {'val': 0.15, 'off': value}
    exp.V23 = {'val': 0.15, 'off': value}
    exp.Vgt = 0
    exp.Vrs = 0.6
    exp.tconst = 0.1
    exp.comment = f'Tmxc={T6}mK / gain=+40dB / sourceatt-20dB / HF_biastees / HF circuit setup / RF-10dB '
    exp.extra_comment = f'RF {rf_onoff}'

set_init(o14)
set_init(o23)
print(tabulate(exp.gettable_ClassAttributes()))

SignalCore10003410 : Set output state to True
-------------  ----------------------------------------------------------------------------------
V14            0.15 / off=-0.3
V23            0.15 / off=-0.3
Vac            0.03
Vch            0.15
Vgt            0
Vrs            0.6
comment        Tmxc=17mK / gain=+40dB / sourceatt-20dB / HF_biastees / HF circuit setup / RF-10dB
comment2       RF True / RF-10dB / V14=V23=Vch+step
extra_comment  RF True
fac            3,000,000.0
hf_freq        5,500,000,000
hf_pow         -10
sa_freq        5,500,000,000
tconst         0.1
-------------  ----------------------------------------------------------------------------------


In [119]:
exp.instr_init()

instruments are initialized


In [120]:
exp.sweep_params(
        var     =   ['Vch', 'V23', 'V14'],
        s1      =   0.15,
        s2      =   2.0,
        num     =   101,
        scale   =   'linear',
        offset  =   [0, 0, 0]
    )

In [121]:
exp_name = 'sweep_16'

o14_list = np.linspace(-0.3, 0.1, num=4, endpoint=True)
step_dict = {
    'step vars': ['o14', 'o23'],
    'step lists': [o14_list]
}

exp.create_file_outer_loop(exp_name, step_dict)

for i in range(len(o14_list)):
    clear_output(wait=True)
    print('step progress ' + str(i) + '/' + str(len(o14_list)))
    
    set_init(o14_list[i])
    exp.instr_init()
    exp.sweep_params(
        var     =   ['Vch', 'V23', 'V14'],
        s1      =   0.15,
        s2      =   2.0,
        num     =   101,
        scale   =   'linear',
        offset  =   [0, o14_list[i], o14_list[i]]
    )
    exp.run_1D(exp_name, savedata=False)
    exp.update_file_outer_loop()
print('experiment is successfully finished')

AttributeError: 'experiment' object has no attribute 'create_file_outer_loop'